In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("YourAppName")
    .config("spark.master", "local[*]")
    .config("spark.driver.extraClassPath", "/spark/jars/sqlite-jdbc-3.36.0.3.jar")
    .config("spark.jars", "/spark/jars/sqlite-jdbc-3.36.0.3.jar")
    .getOrCreate()
)

24/03/26 08:53:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/26 08:53:21 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## CSV 파일

### CSV 파일 읽기

In [2]:
from pyspark.sql.types import LongType, StringType, StructField, StructType

# 스키마 정의
myManualSchema = StructType(
    [
        StructField("DEST_COUNTRY_NAME", StringType(), True),
        StructField("ORIGIN_COUNTRY_NAME", StringType(), True),
        StructField("count", LongType(), False),
    ]
)

# CSV 파일 읽기
df = (
    spark.read.format("csv")
    .option("header", "true")
    .option("mode", "FAILFAST")
    .schema(myManualSchema)
    .load("/data/flight-data/csv/2010-summary.csv")
)

df.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



In [3]:
from pyspark.sql.types import LongType, StringType, StructField, StructType

# 스키마 정의
myManualSchema = StructType(
    [
        StructField("DEST_COUNTRY_NAME", StringType(), True),
        StructField("ORIGIN_COUNTRY_NAME", LongType(), True),  # 잘못된 데이터 포맷 설정
        StructField("count", LongType(), False),
    ]
)

# CSV 파일 읽기 -> 스파크 잡은 시작하자마자 종료된다. [지연 연산 특성]
df = (
    spark.read.format("csv")
    .option("header", "true")
    .option("mode", "FAILFAST")
    .schema(myManualSchema)
    .load("/data/flight-data/csv/2010-summary.csv")
)

df.show(5)

24/03/26 08:53:33 ERROR Executor: Exception in task 0.0 in stage 1.0 (TID 1)
org.apache.spark.SparkException: Encountered error while reading file file:///data/flight-data/csv/2010-summary.csv. Details:
	at org.apache.spark.sql.errors.QueryExecutionErrors$.cannotReadFilesError(QueryExecutionErrors.scala:877)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:307)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:125)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:760)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$getByteArrayRdd$1(SparkPlan.scala:388)
	at org.

Py4JJavaError: An error occurred while calling o54.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 1) (7e98ef8cae67 executor driver): org.apache.spark.SparkException: Encountered error while reading file file:///data/flight-data/csv/2010-summary.csv. Details:
	at org.apache.spark.sql.errors.QueryExecutionErrors$.cannotReadFilesError(QueryExecutionErrors.scala:877)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:307)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:125)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.exe

### CSV 파일 쓰기

In [5]:
csvFile = (
    spark.read.format("csv")
    .option("header", "true")
    .option("mode", "FAILFAST")
    .option("inferSchema", "true")
    .load("/data/flight-data/csv/2010-summary.csv")
)

In [6]:
csvFile.write.format("csv").mode("overwrite").option("sep", "\t").save(
    "/tmp/my-tsv-file.tsv"
)

In [7]:
%ls /tmp/my-tsv-file.tsv/

part-00000-0ac772ec-d9c1-446c-8a8f-777e7faa4fe4-c000.csv  _SUCCESS


## JSON 파일

### JSON 파일 읽기

In [8]:
spark.read.format("json").option("mode", "FAILFAST").option("inferSchema", "true").load(
    "/data/flight-data/json/2010-summary.json"
).show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



### JSON 파일 쓰기

In [9]:
csvFile.write.format("json").mode("overwrite").save("/tmp/my-json-file.json")

In [10]:
%ls /tmp/my-json-file.json/

part-00000-89ece51c-ae79-4c2a-b7d6-0a2993757640-c000.json  _SUCCESS


## 파케이 파일

- 파케이
    - 다양한 스토리지 최적화 기술을 제공하는 오픈소스로 만들어진 컬럼 기반의 데이터 저장 방식
    - 분석 워크로드에 최적화
    - 저장소 공간을 절약할 수 있고, 전체 파일 대신 개별 컬럼을 읽을 수 있음
    - 컬럼 기반의 압축 기능 제공
    - 스파크와 잘 호환되기 때문에 스파크의 기본 파일 포맷이기도 함
    - 읽기 연산 시 JSON이나 CSV보다 훨씬 효율적으로 동작하므로 장기 저장용 데이터는 파케이 포맷으로 저장하는 것이 좋음
    - 복합 데이터 타입 지원 (배열, 맵, 구조체, ...) <- CSV에서는 배열 사용 불가

### 파케이 파일 읽기

In [25]:
spark.read.format("parquet").load(
    "/data/flight-data/parquet/2010-summary.parquet"
).show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



### 파케이 파일 쓰기

In [12]:
csvFile.write.format("parquet").mode("overwrite").save("/tmp/my-parquet-file.parquet")

In [13]:
%ls /tmp/my-parquet-file.parquet/

part-00000-85131749-758a-42f8-858f-882840db18c0-c000.snappy.parquet  _SUCCESS


## ORC 파일

- ORC
    - 하둡 워크로드를 위해 설계된 자기 기술적 \[self-describing\]
    - 데이터 타입을 인식할 수 있는 컬럼 기반의 파일 포맷
    - 대규모 스트리밍 읽기에 최적화
    - 필요한 로우를 신속하게 찾아낼 수 있는 기능이 통합되어 있음
    - 파케이와 매우 유사하나, 파케이는 스파크에 최적화 되어 있고 ORC는 Hive에 최적화 되어 있다는 차이가 있음

### ORC 파일 읽기

In [14]:
spark.read.format("orc").load("/data/flight-data/orc/2010-summary.orc").show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



### ORC 파일 쓰기

In [15]:
csvFile.write.format("orc").mode("overwrite").save("/tmp/my-orc-file.orc")

In [16]:
%ls /tmp/my-orc-file.orc/

part-00000-57b5b200-6beb-4e34-93af-9bbcc3031e44-c000.snappy.orc  _SUCCESS


## SQL 데이터베이스

- SQLite
    - 강력하고 빠르며 이해하기 쉬움
    - 설치와 실행이 쉬움
    - 데이터베이스 접속 시 사용자 인증이 필요 없음
    - 쓰기 연산 시 전체 데이터베이스에 락을 설정하므로 분산 환경에 적합하지 않다.

In [17]:
driver = "org.sqlite.JDBC"
path = "/data/flight-data/jdbc/my-sqlite.db"
url = "jdbc:sqlite:" + path
tablename = "flight_info"

### [SQLite] SQL 테이블 읽어서 DataFrame 만들기

In [20]:
dbDataFrame = (
    spark.read.format("jdbc")
    .option("url", url)
    .option("dbtable", tablename)
    .option("driver", driver)
    .load()
)

오류가 날 경우 `sqlite-jdbc` jar 파일을 설치한 후 스파크 세션을 시작할 때 config 옵션으로 해당 jdbc를 지정해야 한다.  
wget https://github.com/xerial/sqlite-jdbc/releases/download/3.36.0.3/sqlite-jdbc-3.36.0.3.jar -O /$SPARK_HOME/jars/sqlite-jdbc-3.36.0.3.jar

In [22]:
dbDataFrame.select("DEST_COUNTRY_NAME").distinct().show(5)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|         Anguilla|
|           Russia|
|         Paraguay|
|          Senegal|
|           Sweden|
+-----------------+
only showing top 5 rows



### 쿼리 푸시다운  
  
  DataFrame을 만들기 전에 데이터베이스 자체에서 데이터를 필터링하도록 한다.

In [23]:
# 해당 필터에 대한 처리를 데이터베이스로 위임
dbDataFrame.filter("DEST_COUNTRY_NAME in ('Anguilla', 'Sweden')").explain()

== Physical Plan ==
*(1) Scan JDBCRelation(flight_info) [numPartitions=1] [DEST_COUNTRY_NAME#169,ORIGIN_COUNTRY_NAME#170,count#171] PushedFilters: [*In(DEST_COUNTRY_NAME, [Anguilla,Sweden])], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:decimal(20,0)>


In [30]:
# 전체 쿼리를 데이터베이스에 전달해 결과를 DataFrame으로 받을 수도 있다.
# 테이블에서 'DEST_COUNTRY_NAME' 열의 고유한 값을 모두 선택하여 반환
pushdownQuery = """(SELECT DISTINCT(DEST_COUNTRY_NAME) FROM flight_info)
  AS flight_info"""

dbDataFrame = (
    spark.read.format("jdbc")
    .option("url", url)
    .option("dbtable", pushdownQuery)
    .option("driver", driver)
    .load()
)

dbDataFrame.show(5)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|            Egypt|
|Equatorial Guinea|
|       Costa Rica|
|          Senegal|
+-----------------+
only showing top 5 rows



In [31]:
dbDataFrame.explain()

== Physical Plan ==
*(1) Scan JDBCRelation((SELECT DISTINCT(DEST_COUNTRY_NAME) FROM flight_info)
  AS flight_info) [numPartitions=1] [DEST_COUNTRY_NAME#223] PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


### 데이터베이스 병렬로 읽기

In [37]:
dbDataFrame = (
    spark.read.format("jdbc")
    .option("url", url)
    .option("dbtable", tablename)
    .option("driver", driver)
    .option("numPartitions", 10)
    .load()
)  # 읽기 및 쓰기용 동시 작업 수를 제한할 수 있는 최대 파티션 수

### SQL 데이터베이스 쓰기

In [39]:
props = {"driver": "org.sqlite.JDBC"}
newPath = "jdbc:sqlite://tmp/my-sqlite.db"
csvFile.write.jdbc(newPath, tablename, mode="overwrite", properties=props)

2024-03-21 09:54:35 WARN  JdbcUtils:66 - Requested isolation level 1 is not supported; falling back to default isolation level 8


In [41]:
spark.read.jdbc(newPath, tablename, properties=props).count()

255

In [42]:
csvFile.write.jdbc(newPath, tablename, mode="append", properties=props)

2024-03-21 09:56:53 WARN  JdbcUtils:66 - Requested isolation level 1 is not supported; falling back to default isolation level 8


In [43]:
spark.read.jdbc(newPath, tablename, properties=props).count()

510

## 텍스트 파일

### 텍스트 파일 쓰기

In [ ]:
# 문자열 컬럼이 하나만 존재해야 한다.
csvFile.select("DEST_COUNTRY_NAME").write.text("/tmp/simple-text-file.txt")

In [ ]:
# 데이터를 저장할 때 파티셔닝 작업을 수행하면 더 많은 컬럼을 저장할 수 있다.
csvFile.limit(10).select("DEST_COUNTRY_NAME", "count").write.partitionBy("count").text(
    "/tmp/five-csv-files2py.csv"
)

## 고급 I / O 개념  
  
  쓰기 작업 전에 파티션 수를 조절함으로써 병렬로 처리할 파일 수를 제어할 수 있다.  
  또한 버켓팅과 파티셔닝을 조절함으로써 데이터의 저장 구조를 제어할 수 있다.  
    
  특정 파일 포맷은 기본적으로 분할을 지원하므로, 스파크에서 전체 파일이 아닌 쿼리에 필요한 부분만 읽을 수 있다.  
  HDFS 같은 시스템을 사용한다면 분할된 파일을 여러 블록으로 나누어 분산 저장하기 때문에 훨씬 더 최적화할 수 있다.  
  추천하는 파일 포맷과 압축 방식은 '파케이' 파일 포맷과 'GZIP' 압축 방식이다.

### 병렬로 데이터 읽기  
  
  여러 익스큐터가 같은 파일을 동시에 읽을 수는 없지만 여러 파일을 동시에 읽을 수는 있다.  
  폴더의 개별 파일은 DataFrame의 파티션이 된다.  
  따라서 사용 가능한 익스큐터를 이용해 병렬로 파일을 읽는다.

### 병렬로 데이터 쓰기  
  
  파일 또는 데이터의 수는 데이터를 쓰는 시점에 DataFrame이 가진 파티션 수에 따라 달라질 수 있다.  
  기본적으로 데이터 파티션 당 하나의 파일  
  다음 코드는 폴더 안에 5개의 파일을 생성한다.

In [ ]:
csvFile.repartition(5).write.format("csv").save("/tmp/multiple.csv")

In [57]:
%ls /tmp/multiple.csv

part-00000-2d4e5eaa-e056-4c1a-9a4c-fecf7ecc415e-c000.csv
part-00001-2d4e5eaa-e056-4c1a-9a4c-fecf7ecc415e-c000.csv
part-00002-2d4e5eaa-e056-4c1a-9a4c-fecf7ecc415e-c000.csv
part-00003-2d4e5eaa-e056-4c1a-9a4c-fecf7ecc415e-c000.csv
part-00004-2d4e5eaa-e056-4c1a-9a4c-fecf7ecc415e-c000.csv
_SUCCESS


### 파티셔닝  
  
  파티셔닝: 어떤 데이터를 어디에 저장할 것인지 제어할 수 있는 기능  
  데이터를 읽을 때 전체 데이터셋을 스캔하지 않고 필요한 컬럼의 데이터만 읽을 수 있게 한다.  
  모든 파일 기반의 데이터소스에서 지원

In [58]:
csvFile.limit(10).write.mode("overwrite").partitionBy("DEST_COUNTRY_NAME").save(
    "/tmp/partitioned-files.parquet"
)

In [59]:
%ls /tmp/partitioned-files.parquet

'DEST_COUNTRY_NAME=Costa Rica'/         'DEST_COUNTRY_NAME=Senegal'/
'DEST_COUNTRY_NAME=Egypt'/              'DEST_COUNTRY_NAME=United States'/
'DEST_COUNTRY_NAME=Equatorial Guinea'/   _SUCCESS


In [60]:
%ls /tmp/partitioned-files.parquet/DEST_COUNTRY_NAME=Senegal/

part-00000-99767eca-0928-4e6a-9ca3-010f42e63ad6.c000.snappy.parquet


### 버켓팅  
  
  버켓팅: 각 파일에 저장된 데이터를 제어할 수 있는 또 다른 파일 조직화 기법  
  동일한 버킷 ID를 가진 데이터가 하나의 물리적 파티션에 모두 모여 있기 때문에 데이터를 읽을 때 셔플을 피할 수 있다.  
    
  특정 컬럼을 파티셔닝하면 수억 개의 디렉터리가 만들어질 수도 있으므로 데이터를 버켓팅할 수 있는 방법을 찾아야 한다.

In [69]:
# 버켓팅은 스파크 관리 테이블에서만 사용할 수 있다.
numberBuckets = 10
columnToBucketBy = "count"

csvFile.write.format("parquet").mode("overwrite").bucketBy(
    numberBuckets, columnToBucketBy
).saveAsTable("bucketedFiles")

In [72]:
%ls spark-warehouse/bucketedfiles/

part-00000-39450f84-30cb-47de-ad6b-8a4ec6c670a7_00000.c000.snappy.parquet
part-00000-39450f84-30cb-47de-ad6b-8a4ec6c670a7_00001.c000.snappy.parquet
part-00000-39450f84-30cb-47de-ad6b-8a4ec6c670a7_00002.c000.snappy.parquet
part-00000-39450f84-30cb-47de-ad6b-8a4ec6c670a7_00003.c000.snappy.parquet
part-00000-39450f84-30cb-47de-ad6b-8a4ec6c670a7_00004.c000.snappy.parquet
part-00000-39450f84-30cb-47de-ad6b-8a4ec6c670a7_00005.c000.snappy.parquet
part-00000-39450f84-30cb-47de-ad6b-8a4ec6c670a7_00006.c000.snappy.parquet
part-00000-39450f84-30cb-47de-ad6b-8a4ec6c670a7_00007.c000.snappy.parquet
part-00000-39450f84-30cb-47de-ad6b-8a4ec6c670a7_00008.c000.snappy.parquet
part-00000-39450f84-30cb-47de-ad6b-8a4ec6c670a7_00009.c000.snappy.parquet
_SUCCESS


### 파일 크기 관리

In [71]:
# 파일당 최대 5,000 개의 로우를 포함하도록 설정
df.write.option("maxRecordPerFile", 5000)